In [ ]:
#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"

#r "nuget:Deedle"
#r "nuget:Deedle.DotNet.Interactive.Extension,0.1.0-alpha8"
#r "nuget:MathNet.Numerics.FSharp"

#r "nuget: Plotly.NET, 2.0.0-preview.18"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.18"


Restore sources https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json Installed Packages Deedle, 2.5.0 Deedle.DotNet.Interactive.Extension, 0.1.0-alpha8 MathNet.Numerics.FSharp, 5.0.0 Plotly.NET, 2.0.0-preview.18 Plotly.NET.Interactive, 2.0.0-preview.18

In [ ]:
open Deedle

type Model = ObjectSeries<string> -> double

let df = Frame.ReadCsv("/workspaces/transport tycoon/23/s02e03_train.csv")
df

,TRANSPORT,TIME,A,B,SPEED
,System.String,System.DateTime,System.String,System.String,System.Decimal
0,CARGO_15100,11/25/1859 10:16:40,Steamdrift,Rustport,74.09
1,CARGO_15101,11/26/1859 03:58:07,Rustport,Irondale,73.6
2,CARGO_15101,11/26/1859 13:02:07,Irondale,Leverstorm,69.37
3,CARGO_15102,11/26/1859 19:55:34,Leverstorm,Copperhold,82.58
4,CARGO_15102,11/27/1859 09:23:42,Copperhold,Cogburg,77.73
5,CARGO_15103,11/28/1859 02:30:18,Cogburg,Steamdrift,74.17
6,CARGO_15104,11/28/1859 19:45:01,Steamdrift,Cogburg,73.59
7,CARGO_15104,11/29/1859 08:41:30,Cogburg,Irondale,79.9
8,CARGO_15104,11/29/1859 14:59:00,Irondale,Gizbourne,83.6


In [ ]:
open Plotly.NET

let ordered =
    df
    |> Frame.sortRowsBy "TIME" (fun v -> v)

let plotAll (frame: Frame<'A, 'B>) x =
    let speed = frame?SPEED |> Series.values
    let time = frame.GetColumn x |> Series.values

    Chart.Point(
        x=time,
        y=speed
    )
    |> Chart.withYAxisStyle "Speed"

let withPrediction (frame: Frame<'A, 'B>) name graph =
    let prediction = frame?prediction |> Series.values
    let time = frame.GetColumn<DateTime>("TIME") |> Series.values

    Chart.Point(
        x=time,
        y=prediction,
        Name=name
    )
    |> fun c -> [graph; c]
    |> Chart.combine

plotAll ordered

FSI_0172+it@29-1

In [ ]:
// Global avg model
let avgTime = df?SPEED.Sum() / (double df.RowCount)

let model_GlobalAvg: Model = fun _ -> avgTime

In [ ]:
let evaluate model =
    let dfTest = Frame.ReadCsv("/workspaces/transport tycoon/23/s02e03_test.csv")

    dfTest?prediction <-
        dfTest.Rows 
        |> Series.mapValues (fun row -> model row)
    dfTest?error <-
        dfTest.Rows
        |> Series.mapValues (fun row -> Math.Pow(row?SPEED - row?prediction, 2))
    let mse = dfTest?error.Sum() / (double dfTest.RowCount)
    dfTest, mse

let eval_GlobalAvg, mse_GlobalAvg = evaluate model_GlobalAvg
eval_GlobalAvg

,TRANSPORT,TIME,A,B,SPEED,prediction,error
,System.String,System.DateTime,System.String,System.String,System.Decimal,System.Double,System.Double
0,CARGO_17490,12/13/1859 10:56:05,Gizbourne,Rustport,72.17,77.17352508178855,25.03526324408714
1,CARGO_17491,12/14/1859 03:18:00,Rustport,Gizbourne,74.55,77.17352508178855,6.882883854773655
2,CARGO_17492,12/14/1859 15:18:37,Gizbourne,Leverstorm,72.1,77.17352508178855,25.740656755537614
3,CARGO_17493,12/15/1859 00:56:21,Leverstorm,Irondale,65.32,77.17352508178855,140.5060568645905
4,CARGO_17494,12/15/1859 05:17:14,Irondale,Copperhold,79.35,77.17352508178855,4.737043069603494
5,CARGO_17495,12/15/1859 10:33:25,Copperhold,Irondale,65.47,77.17352508178855,136.97249934005382
6,CARGO_17495,12/15/1859 17:25:31,Irondale,Gizbourne,76.58,77.17352508178855,0.3522720227121121
7,CARGO_17496,12/16/1859 00:22:25,Gizbourne,Irondale,75.7,77.17352508178855,2.171276166659957
8,CARGO_17497,12/16/1859 06:23:09,Irondale,Gizbourne,87.49,77.17352508178855,106.42965473808574


In [ ]:
$"MSE is {mse_GlobalAvg}"

MSE is 60.87872530180129

In [ ]:
// Per-Journey avg model A->B
let groupedByJourney: Frame<(string * (string * int)), string> =
    df
    |> Frame.groupRowsBy "A"
    |> Frame.groupRowsBy "B"
    
let journeyAvg = groupedByJourney?SPEED
                |> Series.applyLevel (fun (a, (b, _)) -> (a, b)) (fun s -> s.Sum() / (double s.ValueCount))
                |> Series.sortByKey
                |> Series.map (fun (a,b) v -> (a,b,v))
                |> Series.values
                |> Frame.ofValues
journeyAvg

,Copperhold,Irondale,Rustport,Steamdrift,Cogburg,Leverstorm,Gizbourne
,System.Double,System.Double,System.Double,System.Double,System.Double,System.Double,System.Double
Cogburg,75.44949494949492,79.76871886120996,74.86824561403509,73.52897435897435,N/A,N/A,N/A
Copperhold,N/A,75.15352941176471,N/A,N/A,77.2961616161616,75.25553719008262,N/A
Gizbourne,N/A,81.81487179487183,79.50499999999998,N/A,N/A,76.73847826086958,N/A
Irondale,75.06359116022095,N/A,78.35572289156624,N/A,80.0817894736842,74.70814814814814,81.24081818181813
Leverstorm,74.50417322834645,72.00929577464791,N/A,N/A,N/A,N/A,72.40650000000001
Rustport,N/A,81.29276470588238,N/A,81.26016666666665,78.18102040816328,N/A,77.93559322033896
Steamdrift,N/A,N/A,81.98290909090908,N/A,73.68425992779785,N/A,N/A


In [ ]:
let model_JourneyAvg: Model = fun r ->
    let a = r.GetAs<string>("A")
    let b = r.GetAs<string>("B")
    journeyAvg.[a,b] :?> double

let eval_JourneyAvg, mse_JourneyAvg = evaluate model_JourneyAvg
eval_JourneyAvg

,TRANSPORT,TIME,A,B,SPEED,prediction,error
,System.String,System.DateTime,System.String,System.String,System.Decimal,System.Double,System.Double
0,CARGO_17490,12/13/1859 10:56:05,Gizbourne,Rustport,72.17,77.93559322033896,33.24206518241854
1,CARGO_17491,12/14/1859 03:18:00,Rustport,Gizbourne,74.55,79.50499999999998,24.55202499999984
2,CARGO_17492,12/14/1859 15:18:37,Gizbourne,Leverstorm,72.1,72.40650000000001,0.09394225000000857
3,CARGO_17493,12/15/1859 00:56:21,Leverstorm,Irondale,65.32,74.70814814814814,88.13732565157748
4,CARGO_17494,12/15/1859 05:17:14,Irondale,Copperhold,79.35,75.15352941176471,17.610365397923808
5,CARGO_17495,12/15/1859 10:33:25,Copperhold,Irondale,65.47,75.06359116022095,92.03699134946962
6,CARGO_17495,12/15/1859 17:25:31,Irondale,Gizbourne,76.58,81.81487179487183,27.40388270874468
7,CARGO_17496,12/16/1859 00:22:25,Gizbourne,Irondale,75.7,81.24081818181813,30.70066612396631
8,CARGO_17497,12/16/1859 06:23:09,Irondale,Gizbourne,87.49,81.81487179487183,32.207080144641175


In [ ]:
$"MSE is {mse_JourneyAvg}"

MSE is 52.551138978521806

In [ ]:
plotAll ordered "TIME"
|> withPrediction eval_GlobalAvg "global avg model"
|> withPrediction eval_JourneyAvg "journey avg model"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
open Plotly.NET

let groupedByJourney: Frame<(string * (string * int)), string> =
    df
    |> Frame.sortRowsBy "TIME" (fun v -> v)
    |> Frame.groupRowsBy "A"
    |> Frame.groupRowsBy "B"

let plotJourneys (frame: Frame<('A * ('A * 'B)), 'C>) (a: 'A) (b: 'A) name =
    let journeys =
        frame
        |> Frame.nest
        |> (fun f -> f.[a])
        |> Frame.nest
        |> (fun f -> f.[b])

    let speed = journeys?SPEED |> Series.values
    let time = journeys.GetColumn<DateTime>("TIME") |> Series.values

    Chart.Point(
        x=time,
        y=speed,
        Name=name
    )
    |> Chart.withYAxisStyle "Speed"
    |> Chart.withSize (1024, 800)


[
    plotJourneys groupedByJourney "Steamdrift" "Rustport" "Steamdrift->Rustport";
    plotJourneys groupedByJourney "Rustport" "Steamdrift" "Rustport->Steamdrift";
    plotJourneys groupedByJourney "Steamdrift" "Cogburg" "Steamdrift->Cogburg";
    plotJourneys groupedByJourney "Cogburg" "Steamdrift" "Cogburg->Steamdrift";
] |> Chart.Grid(2,2)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let groupedByJourneyDay = 
    df
    |> Frame.groupRowsUsing (fun _ row -> row.GetAs<DateTime>("TIME").DayOfWeek)

let journeyAvgDay = groupedByJourneyDay?SPEED
                    |> Series.applyLevel (fun (a, b) -> a) (fun s -> s.Sum() / (double s.ValueCount))
                    |> Series.sortByKey
journeyAvgDay

A series: 7 values. Key type: System.DayOfWeek Keys Sunday Monday Tuesday Wednesday Thursday Friday Saturday Values 83.10509181969942 81.52198795180713 74.82135902636915 74.20133037694019 73.8344147843942 73.88206349206348 77.65160558464234

In [ ]:
let model_JourneyAvgDay: Model = fun r ->
    let day = r.GetAs<DateTime>("TIME").DayOfWeek
    day |> int |> Series.getAt <| journeyAvgDay

let eval_JourneyAvgDay, mse_JourneyAvgDay = evaluate model_JourneyAvgDay
eval_JourneyAvgDay

,TRANSPORT,TIME,A,B,SPEED,prediction,error
,System.String,System.DateTime,System.String,System.String,System.Decimal,System.Double,System.Double
0,CARGO_17490,12/13/1859 10:56:05,Gizbourne,Rustport,72.17,74.82135902636915,7.029704686709166
1,CARGO_17491,12/14/1859 03:18:00,Rustport,Gizbourne,74.55,74.20133037694019,0.12157050604466695
2,CARGO_17492,12/14/1859 15:18:37,Gizbourne,Leverstorm,72.1,74.20133037694019,4.415589353051631
3,CARGO_17493,12/15/1859 00:56:21,Leverstorm,Irondale,65.32,73.8344147843942,72.49525912071061
4,CARGO_17494,12/15/1859 05:17:14,Irondale,Copperhold,79.35,73.8344147843942,30.421680270609237
5,CARGO_17495,12/15/1859 10:33:25,Copperhold,Irondale,65.47,73.8344147843942,69.96343468539226
6,CARGO_17495,12/15/1859 17:25:31,Irondale,Gizbourne,76.58,73.8344147843942,7.538238176153148
7,CARGO_17496,12/16/1859 00:22:25,Gizbourne,Irondale,75.7,73.88206349206348,3.304893146888438
8,CARGO_17497,12/16/1859 06:23:09,Irondale,Gizbourne,87.49,73.88206349206348,185.17593600403143


In [ ]:
$"MSE is {mse_JourneyAvgDay}"

MSE is 48.871128634190754

In [ ]:
plotAll ordered "TIME"
|> withPrediction eval_JourneyAvgDay "day-of-week model"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let groupedByJourneyAndDay : Frame<(DayOfWeek * (string * (string * int))), string> = 
    ordered
    |> Frame.groupRowsBy "A"
    |> Frame.groupRowsBy "B"
    |> Frame.groupRowsUsing (fun _ row -> row.GetAs<DateTime>("TIME").DayOfWeek)

groupedByJourneyAndDay

,TRANSPORT,TIME,A,B,SPEED
,System.String,System.DateTime,System.String,System.String,System.Decimal
"(Thursday, (Copperhold, (Irondale, 3356)))",CARGO_17290,11/24/1859 12:42:54,Irondale,Copperhold,73.17
"(Thursday, (Copperhold, (Irondale, 1371)))",CARGO_16000,11/24/1859 12:51:29,Irondale,Copperhold,71.01
"(Thursday, (Copperhold, (Irondale, 505)))",CARGO_15430,11/24/1859 13:22:01,Irondale,Copperhold,64.28
"(Thursday, (Copperhold, (Irondale, 2606)))",CARGO_16810,11/24/1859 13:30:58,Irondale,Copperhold,62.54
"(Thursday, (Copperhold, (Irondale, 2150)))",CARGO_16510,11/24/1859 19:44:37,Irondale,Copperhold,72.66
"(Thursday, (Copperhold, (Irondale, 2199)))",CARGO_16540,11/24/1859 19:47:12,Irondale,Copperhold,67.1
"(Thursday, (Copperhold, (Irondale, 1430)))",CARGO_16030,11/24/1859 19:51:54,Irondale,Copperhold,72.16
"(Thursday, (Copperhold, (Irondale, 3496)))",CARGO_17380,11/24/1859 20:20:47,Irondale,Copperhold,64.83
"(Thursday, (Copperhold, (Irondale, 1178)))",CARGO_15880,11/24/1859 20:33:35,Irondale,Copperhold,67.93


In [ ]:
let journeyAndDayAvg = groupedByJourneyAndDay?SPEED
                        |> Series.applyLevel (fun (d, (a, (b, _))) -> (d, a, b)) (fun s -> s.Sum() / (double s.ValueCount))
                        |> Series.sortByKey
                        |> Series.map (fun (d,a,b) v -> (d,a,b,v))

journeyAndDayAvg |> Series.values

index,Item1,Item2,Item3,Item4
0,Sunday,Cogburg,Copperhold,82.18583333333332
1,Sunday,Cogburg,Irondale,85.18435897435899
2,Sunday,Cogburg,Rustport,84.83800000000001
3,Sunday,Cogburg,Steamdrift,80.27333333333335
4,Sunday,Copperhold,Cogburg,82.6126315789474
5,Sunday,Copperhold,Irondale,81.34576923076924
6,Sunday,Copperhold,Leverstorm,81.0340909090909
7,Sunday,Gizbourne,Irondale,87.62500000000001
8,Sunday,Gizbourne,Leverstorm,83.69500000000001
9,Sunday,Gizbourne,Rustport,88.09499999999998


In [ ]:
let model_JourneyAndDayAvg: Model = fun r ->
    let day = r.GetAs<DateTime>("TIME").DayOfWeek
    let a = r.GetAs<string>("A")
    let b = r.GetAs<string>("B")

    let arg = (day, a, b)

    journeyAndDayAvg.Get arg
    |> fun (_, _, _, v) -> v

let eval_JourneyAndDayAvg, mse_JourneyAndDayAvg = evaluate model_JourneyAndDayAvg
eval_JourneyAndDayAvg


,TRANSPORT,TIME,A,B,SPEED,prediction,error
,System.String,System.DateTime,System.String,System.String,System.Decimal,System.Double,System.Double
0,CARGO_17490,12/13/1859 10:56:05,Gizbourne,Rustport,72.17,76.687,20.403288999999962
1,CARGO_17491,12/14/1859 03:18:00,Rustport,Gizbourne,74.55,73.68888888888888,0.7415123456790178
2,CARGO_17492,12/14/1859 15:18:37,Gizbourne,Leverstorm,72.1,73.852,3.0695040000000335
3,CARGO_17493,12/15/1859 00:56:21,Leverstorm,Irondale,65.32,66.175,0.7310250000000068
4,CARGO_17494,12/15/1859 05:17:14,Irondale,Copperhold,79.35,72.1812,51.39169343999986
5,CARGO_17495,12/15/1859 10:33:25,Copperhold,Irondale,65.47,71.23,33.17760000000006
6,CARGO_17495,12/15/1859 17:25:31,Irondale,Gizbourne,76.58,76.92340909090909,0.11792980371900848
7,CARGO_17496,12/16/1859 00:22:25,Gizbourne,Irondale,75.7,79.388275862069,13.603378834720761
8,CARGO_17497,12/16/1859 06:23:09,Irondale,Gizbourne,87.49,78.40120000000002,82.6062854399996


In [ ]:
$"MSE is {mse_JourneyAndDayAvg}"

MSE is 45.252224009655706

In [ ]:
plotAll ordered "TIME"
|> withPrediction eval_JourneyAndDayAvg "day-of-week+journey model"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let time = 
    ordered
    |> Frame.mapRows (fun _ row -> row.GetAs<DateTime>("TIME").TimeOfDay.TotalHours)
    |> Frame.addCol "timeofday" <| ordered


plotAll time "timeofday"

Error: System.InvalidCastException: Object must implement IConvertible.
   at System.Convert.ChangeType(Object value, Type conversionType, IFormatProvider provider)
   at Deedle.Internal.Convert.convertType[T](ConversionKind conversionKind, Object value) in C:\FSharp\zyzhu\Deedle\src\Deedle\Common\Common.fs:line 1440
   at Deedle.VectorHelpers.convertType@273-3.Invoke(T value) in C:\FSharp\zyzhu\Deedle\src\Deedle\Vectors\VectorHelpers.fs:line 273
   at Deedle.F# Vector extensions (core).IVector-1-Select@127.Invoke(IVectorLocation _arg1, OptionalValue`1 input)
   at <StartupCode$Deedle>.$ArrayVector.data@380-6.Invoke(Int32 i, T v) in C:\FSharp\zyzhu\Deedle\src\Deedle\Vectors\ArrayVector.fs:line 380
   at Microsoft.FSharp.Collections.ArrayModule.MapIndexed[T,TResult](FSharpFunc`2 mapping, T[] array) in D:\a\_work\1\s\src\fsharp\FSharp.Core\array.fs:line 351
   at Deedle.Vectors.ArrayVector.ArrayVector`1.Deedle.IVector<'T>.Select[TNewValue](FSharpFunc`2 f) in C:\FSharp\zyzhu\Deedle\src\De